In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [25]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import tensorflow as tf
import nltk
import re
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [12]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
from datasets import load_dataset
#load IMDB data
df = load_dataset("stanfordnlp/imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
df #we are not using unsupervised dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
train_data = df["train"]
test_data = df["test"]
train_df = pd.DataFrame(df["train"])
test_df = pd.DataFrame(df["test"])
test_df , _= train_test_split(test_df, test_size=0.8, random_state=42,)

In [ ]:
tf.__version__

'2.15.0'

In [13]:
#preprocessing data

happy_emojis = [':-D', ':)', ':P', ';)', ';-)']
sad_emojis = [':(', ';(', ':—',':-(']

#define expanding contraction
dict = {
    'don\'t': 'do not',
    'didn\'t': 'did not',
    'thx': 'thanks',
    'thnx': 'thanks',
    'shouldn\'t': 'should not',
    'wouldn\'t': 'would not',
    'couldn\'t': 'could not',
    'haven\'t': 'have not',
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "isn't" : "is not"

}

#define pre-process function
def preprocess(text):
    text = text.lower() #lowercase the rext
    text = re.sub('<br />', '', text) #remove hyperlink
    text = re.sub(r'http\S+', '', text) #remove webaddress
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)

    # Replace emotion patterns
    for emoji in happy_emojis:
        text = text.replace(emoji, 'happy')
    for emoji in sad_emojis:
        text = text.replace(emoji, 'sad')


    puntuation_pattern = re.compile(r'\b(' + '|'.join(dict.keys()) + r')\b')
    text = puntuation_pattern.sub(lambda x: dict[x.group(0)], text) #replace short forms with full forms

    text = text.replace('?', ' questionmark') #define ? and ! to a word which could define emotions of positive and
    text = text.replace('!', ' exclamatorymark')

    text = re.sub('[^a-zA-Z]', ' ', text) #remove non alphabatical character

    tokens = re.findall(r'\b\w+\b', text) # text to tokens, split on space


    # lemitization and remove stop words
    lemmatize = WordNetLemmatizer()
    stopWords = set(stopwords.words('english'))
    stopWords = set(stopWords)
    stopWords.discard('not') # not including not in stop word
    stopWords.discard('but') #not including but in stop word

    text_lemma = [lemmatize.lemmatize(word) for word in tokens if word not in stopWords]


    text = ' '.join(text_lemma) #convert to original text by joining with space

    return text

In [14]:
x_final = [preprocess(text) for text in train_df['text']]

In [15]:
x_final[0]

'rented curious yellow video store controversy surrounded first released also heard first seized u custom ever tried enter country therefore fan film considered controversial really see plot centered around young swedish drama student named lena want learn everything life particular want focus attention making sort documentary average swede thought certain political issue vietnam war race issue united state asking politician ordinary denizen stockholm opinion politics sex drama teacher classmate married men kill curious yellow year ago considered pornographic really sex nudity scene far even not shot like cheaply made porno countryman mind find shocking reality sex nudity major staple swedish cinema even ingmar bergman arguably answer good old boy john ford sex scene film commend filmmaker fact sex shown film shown artistic purpose rather shock people make money shown pornographic theater america curious yellow good film anyone wanting study meat potato pun intended swedish cinema but 

In [17]:
#one-hot of the vocabulary encoding required for dense embeddin
vocab_size=5000 #Vocabulary size
onehot_vocab=[one_hot(words,vocab_size)for words in x_final]


In [18]:
review_length =600 # from EDA length of sentence goes upto 800 so we took 600 as an average length as the data is uniformly distributed
embedded_reviews =pad_sequences(onehot_vocab,padding='pre',maxlen=review_length) # review embedding using one hot encoding and pre-padded
print(embedded_reviews[:2])

[[   0    0    0 ... 4247 4494 3873]
 [   0    0    0 ...  180 3751 2622]]


In [23]:
embedding_features_vector =128
model=Sequential()
model.add(Embedding(vocab_size,embedding_features_vector,input_length=review_length))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))) # add regularisation L1 and L2 to overcome overfitting
model.add(Dropout(0.3)) # Dropout with p value 0.3
model.add(Dense(1, activation='sigmoid')) # output layer have one neuron as we are using binary crossentropy
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 600, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 100)               91600     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 64)                6464      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 738129 (2.82 MB)
Trainable params: 738129 

In [24]:
X_final=np.array(embedded_reviews) # convert to array which will be required in the format while training
y_final=np.array(train_df['label'])

In [ ]:
# split the train data into 80% train and 20% test
X_train, X_val, y_train, y_val = train_test_split(X_final, y_final, test_size=0.20, random_state=42)

In [ ]:
batch_size = 32 # define batch size and number of epochs
epochs = 5

In [ ]:
#train model
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=10,batch_size=32)

Epoch 1/10
625/625 [==============================] - 452s 719ms/step - loss: 1.4217 - accuracy: 0.6198 - val_loss: 0.6990 - val_accuracy: 0.6518
Epoch 2/10
625/625 [==============================] - 452s 723ms/step - loss: 0.6072 - accuracy: 0.7488 - val_loss: 0.7749 - val_accuracy: 0.6494
Epoch 3/10
625/625 [==============================] - 451s 723ms/step - loss: 0.5453 - accuracy: 0.8001 - val_loss: 0.4901 - val_accuracy: 0.8222
Epoch 4/10
625/625 [==============================] - 453s 725ms/step - loss: 0.4272 - accuracy: 0.8657 - val_loss: 0.4559 - val_accuracy: 0.8416
Epoch 5/10
625/625 [==============================] - 443s 709ms/step - loss: 0.3769 - accuracy: 0.8897 - val_loss: 0.4332 - val_accuracy: 0.8490
Epoch 6/10
625/625 [==============================] - 455s 728ms/step - loss: 0.3330 - accuracy: 0.9095 - val_loss: 0.3927 - val_accuracy: 0.8706
Epoch 7/10
625/625 [==============================] - 455s 728ms/step - loss: 0.2994 - accuracy: 0.9244 - val_loss: 0.4004 -

In [ ]:
x_test = [preprocess(text) for text in test_df['text']]

In [ ]:
onehot_vocab_test=[one_hot(words,vocab_size)for words in x_test]


In [ ]:
embedded_docs_test=pad_sequences(onehot_vocab_test,padding='pre',maxlen=review_length) # perform same for test data

In [ ]:
X_final_test=np.array(embedded_docs_test)

In [ ]:
#make predictions
y_pred = model.predict(X_final_test)

#convert y_pred to (0 or 1)
predicted_labels = (y_pred > 0.5).astype(int)

# save predicted labels to test_df dataframe
test_df['predicted_labels'] = predicted_labels

157/157 [==============================] - 35s 220ms/step


In [ ]:
# confusion matrix for perfromance
confusion_matrix(test_df['label'],test_df['predicted_labels'] )

array([[2117,  345],
       [ 360, 2178]])

In [ ]:
#Accuracy obtained
accuracy_score(test_df['label'],test_df['predicted_sentiment'] )

0.859